In [1]:
## Import all necessary packages

#To define and simulate the model
using DifferentialEquations 

#To use the Bayesian inference framework
using Turing
using Distributions #To use within Turing
using MCMCChains

#To load the data
using DataFrames
using CSV

Turing.setadbackend(:forwarddiff)

:forwarddiff

In [2]:
# Model of differential equations compatible with DifferentialEquations.jl
function SEIR_v!(du, u, p, t)
    
    S, E, I, R, Sv, Iv = u #functions
    
    β, κ, γ, α, μ, N = p #parameters
    
    #Differential equations defining the model
    du[1] = dS = -β*S*Iv/N
    du[2] = dE = β*S*Iv/N - κ*E
    du[3] = dI = κ*E - γ*I
    du[4] = dR = γ*I
    
    du[5] = dSv = -α*Sv*I/N - μ*Sv
    du[6] = dIv = α*Sv*I/N - μ*Iv
    
end

# Basic reproductive number
function R_0(β, α, κ, γ, μ, N, Nv, S0, τ)
   
    return  β*α/ (γ*μ) * (S0/N^2) * (Nv/(μ*τ)) * (1-exp(-μ*τ))
    
end

# Auxiliar function to easily obtain the model results
function get_vars(sol)
   
    S = []
    E = []
    I = []
    R = []
    
    S_V = []
    I_V = []
    
    for item in sol.u
       
        append!(S, item[1])
        append!(E, item[2])
        append!(I, item[3])
        append!(R, item[4])
        
        append!(S_V, item[5])
        append!(I_V, item[6])
        
    end
    
    return S, E, I, R, S_V, I_V
    
end

get_vars (generic function with 1 method)

# Load data 

In [3]:
#For this example, we load the data from the OQDS outbreak in Apulia
df = DataFrame(CSV.File("Data/OQDS_data.csv"))

#This is the time corresponding to the datapoints
t_exp = collect(4:1:10)

#Use the sum of S+E (S_E) I+R (I_R) compartments
df[!, "I_R"] = df[!, "I"] .+ df[!, "R"]

#Define the data to be fitted
fit_data = Float64.(Array(df[!, ["S_E", "I_R"]])[5:end-2, :]);

# Bayesian inference 

In [5]:
#Define fixed initial conditions
N_years = 12

#The number of hosts is known, we assume 1 vector per 2 hosts.
N = 2959
Nv = 2959 * 0.5

t = 365 * N_years

E0 = 0.006 * N #Initial condition comes from White et al work.
I0 = 0.0
S0 = N - I0
R0 = 0

Sv0 = Nv
Iv0 = 0

initial_conditions = [S0, E0, I0, R0, Sv0, Iv0]

time = (0.0, t)

prob1 = ODEProblem(SEIR_v!, initial_conditions, time, parameters)

#Iterations in which to compare model with data points
index_sol = [1461, 1826, 2191, 2556, 2921, 3286, 3651]

@model function fitlv(data, prob1)
    
    dosetimes = [365.0 * i for i in 1 : N_years]

    affect!(integrator) = integrator.u[5] = Nv

    cb = PresetTimeCallback(dosetimes,affect!)
        
    τ_I ~ Truncated(Normal(3.5, 1), 0.01, 10)
    τ_E ~ Truncated(Normal(1.75, 0.5), 0.1, 4)
    
    β ~ Uniform(1e-2, 1e1)
    α ~ Uniform(1e-5, 1e-2)

    μ ~ Truncated(Normal(0.02, 0.0075), 0.01, 0.04) #0.02 #~ Uniform(0.02, 0.04)
    
    σ ~ InverseGamma(10, 1)
    
    γ = (1.0 ./ τ_I) ./ 365.0
    κ = (1.0 ./ τ_E) ./ 365.0
    
    p = [β, κ, γ, α, μ, N]
    
    prob = remake(prob1, p=p)
    
    predicted = solve(prob, RK4(), adaptative=false, dt=1e-1, saveat=1, maxiters=10^9, callback=cb)
    
    S, E, I, R, Sv, Iv = get_vars(predicted)
    
    S_E_fit = (S .+ E) ./ N
    I_R_fit = (I .+ R) ./ N
    
    affected = hcat(S_E_fit, I_R_fit)
    
    for i in 1 : size(data)[1]
    
        data[i, :] ~ MvNormal(affected[index_sol[i], :], σ)
        
    end
        
end

#Define the model to be optimized through Bayesian inference
model = fitlv(fit_data, prob1)

DynamicPPL.Model{typeof(fitlv), (:data, :prob1), (), (), Tuple{Matrix{Float64}, ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, typeof(parameters), ODEFunction{true, typeof(SEIR_v!), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing}, Base.Iterators.Pairs{Union{}, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}, SciMLBase.StandardODEProblem}}, Tuple{}, DynamicPPL.DefaultContext}(:fitlv, fitlv, (data = [0.971036585365854 0.030487804878049; 0.5929878048780487 0.4086495031616979; … ; 0.0148628048780485 0.9971452969738027; 0.02286585365853626 0.9828100299232154], prob1 = ODEProblem{Vector{Float64}, Tuple{Float64, Float64}, true, typeof(parameters), ODEFunction{true, typeof(SEIR_v!), LinearAlgebra.UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT

In [6]:
#Set some initial values for the parameters
params = [3.5, 1.0, 1, 1e-3, 0.02, 0.0725]

#Infer the parameters using 1 Markov Chain MonteCarlo (MCMC)
@time chain = mapreduce(c -> sample(model, NUTS(.65), 10^3, init_params = params), chainscat, 1)

┌ Info: Found initial step size
│   ϵ = 0.025
└ @ Turing.Inference /home/alex/.julia/packages/Turing/Ir2iS/src/inference/hmc.jl:188
Sampling:   7%|██▉                                      |  ETA: 0:08:44┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/alex/.julia/packages/AdvancedHMC/w90s5/src/hamiltonian.jl:47
Sampling:  12%|█████                                    |  ETA: 0:10:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/alex/.julia/packages/AdvancedHMC/w90s5/src/hamiltonian.jl:47
Sampling:  16%|██████▌                                  |  ETA: 0:09:34┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /home/alex/.julia/packages/AdvancedHMC/w90s5/src/hamiltonian.jl:4

893.520727 seconds (10.68 G allocations: 1014.957 GiB, 13.53% gc time, 0.01% compilation time)


Chains MCMC chain (1000×18×1 Array{Float64, 3}):

Iterations        = 501:1:1500
Number of chains  = 1
Samples per chain = 1000
Wall duration     = 853.6 seconds
Compute duration  = 853.6 seconds
parameters        = τ_I, τ_E, β, α, μ, σ
internals         = lp, n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat   e ⋯
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64     ⋯

         τ_I    3.6322    0.9231     0.0292    0.0348   669.0897    0.9990     ⋯
         τ_E    1.2855    0.2788     0.0088    0.0087   918.9727    1.0012     ⋯
           β    3.7344    2.7718     0.0877    0.1422   400.8475    0.9995     ⋯
           α    0.0030    0.0024     0.0001    0.0001   427.7632    1.0108     ⋯
           μ    0.0241    0.0065     0.0002    0.000